# Mathematik für Biologiestudierende II

Sommersemester 2024

16.04.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

# Normalverteilungsannahmen

## konservative Tests

* Der $t$-Test verwendet eine Verteilungsannahme:  Daten müssen normalverteilt sein
* Tests, die auch bei Verletzung der Verteilungsannahmen noch gute Ergebnisse liefern, heißen *konservativ*
* Der $t$-Test ist konservativ

## Q-Q-Plot

* Mit dem Quantil-Quantil-Plot kann man auf graphischem Wege beurteilen, ob Messwerte Realisierungen einer normalverteilten Zufallsvariablen sind
* Man trägt dazu auf der $x$-Achse die Quantile der Standardnormalverteilung und auf der $y$-Achse die Quantile der
    Beobachtungsdaten auf
* Wenn diese Punkte annähernd auf einer Geraden liegen, sind die Daten näherungsweise normalverteilt, ansonsten nicht

* es gibt auch Testverfahren, um auf Normalverteilungsannahmen zu testen
* nicht ganz klar, wie sinnvoll das ist

Beispieldaten aus Lektion 12

In [ ]:
u = "https://www.math.uni-duesseldorf.de/~braun/bio2324/data/schadstoffe.csv"
df = pd.read_csv(u, index_col=0)
df

In [ ]:
import statsmodels.api as sm

In [ ]:
pp = sm.ProbPlot(df.Konzentration)
pp.qqplot();

Wunderbar normalverteilt

Die Daten aus dem synthetischen Medikamentenexperiment aus Lektion 13

In [ ]:
df = pd.read_csv('treatment.csv', index_col=0)

In [ ]:
pp = sm.ProbPlot(df.t0)
pp.qqplot();

Die Daten sind nicht normalverteilt, weil ich oben bei 100 abgeschnitten hatte

### Beispiel Galapagos Inseln

Ein Datensatz zum Buch "Linear Models with Python" von Faraway

In [ ]:
df = pd.read_csv("galapagos.csv", index_col=0)
df

In [ ]:
pp = sm.ProbPlot(df.Area)
pp.qqplot();

# Nicht-parametrische Tests

Beispiel für Situationen, in denen man nicht-parametrische Tests macht:

* Wenn die Verteilungsannahmen nicht erfüllt sind
* Wenn die Stichprobenumfänge zu klein sind
* Wenn die Zufallsvariable diskret ist

### Vergleich zweier Erwartungswerte bzw. zweier Mediane

|Vergeich                 | parametrisch                        |   nicht-parametrisch |
|-------------------------|-------------------------------------|----------------------|
|mit Referenzwert         | t-Test für verbundene Stichproben   | Wilcoxon-Test        |
|vorher-nachher           | t-Test für verbundene Stichproben   | Wilcoxon-Test        |
|verschiedene Populationen| t-Test für unverbundene Stichproben | Mann-Whitney-U-Test  |


## Wilcoxon-Signed-Rank-Test

Den Wilcoxon Test verwendet man zum Vergleich der Mediane verbundener Datensätze, wenn die Normalverteilungsannahme nicht gesichert ist.

Er ist ein Rangtest:  Das bedeutet, dass nur eine Rolle spielt, ob Werte größer sind als andere, aber nicht, um wie viel.

In [ ]:
from scipy import stats

In [ ]:
df = pd.read_csv(u, index_col=0)
df['Referenz'] = 0.08 / 100

* Wir vergleichen die Konzentration mit dem Referenzwert 0.08
* Die Nummer der Messstelle benötigen wir erst in einer späteren Auswertung

In [ ]:
res = stats.wilcoxon(df.Konzentration, df.Referenz, alternative="greater")
res

Zum Vergleich:

In [ ]:
stats.ttest_rel(df.Konzentration, df.Referenz, alternative="greater")

* Der p-Wert ist ein kleines bisschen schlechter
* Dieser Unterschied ist unerheblich
* Im allgemeinen ist nicht klar, ob die unberechtigte Nutzung eines parametrischen Tests den p-Wert verbessert oder verschlechtert

Wir bestimmen jetzt auch noch die Effektstärke gemäß *Cohen's&nbsp;r*

Dazu ist es erforderlich, den Test noch einmal mit `method="approx"` zu rechnen, um die z-Statistik zu bekommen

In [ ]:
res = stats.wilcoxon(df.Konzentration, df.Referenz, alternative="greater", method="approx")
res

In [ ]:
res.zstatistic

In [ ]:
n = df.Konzentration.count()
n

In [ ]:
r = abs(res.zstatistic / np.sqrt(n))
r

### Interpretation der Effektstärke

| r-Wert | Interpretation   |
|--------|------------------|
| 0.1    | geringer Effekt  |
| 0.3    | mittlerer Effekt |
| 0.5    | starker Effekt   |

Wir haben also einen mittleren Effekt beobachtet

# Mann-Whitney-U-Test

Den Mann-Whitney Test verwendet man zum Vergleich der Mediane unverbundener Datensätze, wenn die Normalverteilungsannahme nicht gesichert ist

In [ ]:
df = pd.read_csv('treatment.csv', index_col=0)
dfv = df[df.Treatment=='Verum']
dfp = df[df.Treatment=='Placebo']

In [ ]:
res = stats.mannwhitneyu(dfv.Difference, dfp.Difference, alternative='greater')
res

zum Vergleich

In [ ]:
stats.ttest_ind(dfv.Difference, dfp.Difference, alternative='greater')

In [ ]:
pp = sm.ProbPlot(dfv.Difference)
pp.qqplot();

Auch hier müssen wir für die Effektstärke die Statistik noch einmal rechnen, und zwar mit vertauschten Datensätzen

In [ ]:
res2 = stats.mannwhitneyu(dfp.Difference, dfv.Difference, alternative='less')
res2

Der $p$-Wert ist derselbe, aber die Statistik ist eine andere

Die Formel für die Effektstärke nach Cohen's&nbsp;r ist
$$  r = 1 - \frac{2U}{n_1 \cdot n_2}
$$

wobei $U$ die kleinere der beiden Statistiken ist und $n_1$ und $n_2$ die Stichprobenumfänge in den beiden Populationen sind

In [ ]:
U = res2.statistic
n1 = dfp.Difference.count()
n2 = dfv.Difference.count()

In [ ]:
r = 1 - 2*U/(n1*n2)
r

Ein sehr geringer Effekt